# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data=pd.read_csv('output_data\city_data.csv')
city_data

,Unnamed: 0,City,Latitude,Longitude,Humidity,Wind_Speed,TemperatureK,Cloudiness,Temperature
0,0,Port Alfred,-33.59,26.89,39,3.12,286.64,0,56
1,1,Yatou,37.15,122.38,60,5.11,298.78,46,78
2,2,Guerrero Negro,27.98,-114.06,84,6.87,292.84,0,67
3,3,Ushuaia,-54.80,-68.30,74,2.10,271.15,0,28
4,4,Albany,42.60,-73.97,83,5.12,292.01,100,65
...,...,...,...,...,...,...,...,...,...
564,564,Pánuco,22.05,-98.17,72,5.70,288.15,90,58
565,565,Antoniny,49.81,26.88,89,0.96,285.98,84,55
566,566,Ust-Maya,60.42,134.53,35,2.30,301.65,94,83
567,567,Itagi,-14.16,-40.01,93,1.92,292.19,88,66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
locations = city_data[["Latitude","Longitude"]]

# Fill NaN values and convert to float
rating = city_data["Humidity"].astype(int)


fig = gmaps.figure(center=(0,0),zoom_level=1.5)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating,
                                dissipating=False, max_intensity=100,
                                 point_radius=2,opacity =.8)


# Add layer


fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

#city_data.Temperature<80 & city_data.Temperature>70 & city_data.Wind_Speed<10 & city_data.Cloudiness==0

Best_cities=city_data.loc[(city_data.Temperature<80) & (city_data.Wind_Speed<10) & (city_data.Temperature>70)  & (city_data.Cloudiness==0)]
Best_cities.head()

,Unnamed: 0,City,Latitude,Longitude,Humidity,Wind_Speed,TemperatureK,Cloudiness,Temperature
76,76,Balkanabat,39.51,54.37,70,2.35,299.26,0,78
99,99,Voh,-20.97,164.70,45,3.84,295.84,0,72
195,195,Komsomolets,53.75,62.07,65,2.38,296.81,0,74
199,199,Airai,-8.93,125.41,58,1.61,296.03,0,73
264,264,Laguna,38.42,-121.42,47,2.24,298.48,0,77


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# You can ignor this part the data frame was alreacy saved as "Best_Hotel_Data.csv"

target_radius = 5000
target_type = "Hotel"
target_search = "Hotel"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a parameters dictionary

Hotel_Names=[]
Hotel_Adresses=[]
Hotel_Locations=[]
Hotel_Lat=[]
Hotel_Lng=[]
Best_cities_List=Best_cities['City']
City_names=[]
Country=[]
City_Humidity=[]


# run a request using our params dictionary
print ("Start Of Searching the Hotel in the Cities")
print ("------------------------\n")

i=0
for City in Best_cities_List:
    response=[]
    target_coordinates=str(Best_cities.iloc[i,2])+","+str(Best_cities.iloc[i,3])
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key,
        "keyword": target_search
        }
    try:
        response = requests.get(base_url, params=params).json()
        Hotel_Name=response["results"][0]["name"]
        Hotel_Adress=response["results"][0]["vicinity"]
        Hotel_Location=response["results"][0]['geometry']['location']
        Hotel_lat=response["results"][0]['geometry']['location']['lat']
        Hotel_lng=response["results"][0]['geometry']['location']['lng']
        Contry_code=response["results"][0]["plus_code"]['compound_code']
        
        Contry_code_List = Contry_code.split(',')  # list of words
        Country_name=Contry_code_List[-1]
        
        City_Humidity.append(Best_cities.iloc[i,4])
        Country.append(Country_name)
        City_names.append(City)
        Hotel_Adresses.append(Hotel_Adress)
        Hotel_Locations.append(Hotel_Location)
        Hotel_Names.append(Hotel_Name)
        Hotel_Lat.append(Hotel_lat)
        Hotel_Lng.append(Hotel_lng)

        
        print (f'We found a hotel in {City}')
    except:
        print(f'Could not find any {target_search} in {target_radius} meters of {City}')
    #if i==2: break  #For Testing
    i=i+1
print("------------------------")
print("End Of Search")
# base url

Start Of Searching the Hotel in the Cities
------------------------

We found a hotel in Balkanabat
We found a hotel in Voh
We found a hotel in Komsomolets
Could not find any Hotel in 5000 meters of Airai
We found a hotel in Laguna
We found a hotel in Aswān
We found a hotel in Townsville
We found a hotel in Puerto del Rosario
We found a hotel in Sirte
We found a hotel in Linxia Chengguanzhen
We found a hotel in São João dos Patos
We found a hotel in Diamantino
We found a hotel in Marsaxlokk
We found a hotel in Kuandian
We found a hotel in Noumea
We found a hotel in Kizilskoye
We found a hotel in Acaraú
------------------------
End Of Search


In [7]:
# You can ignor this part the  Best_Hotel data frame was alreacy saved as "Best_Hotel_Data.csv"

Best_Hotel_data = pd.DataFrame({
                                'Country':Country,
                                'Humidity':City_Humidity,
                                'City': City_names,
                               'Name': Hotel_Names,
                               'Latitude': Hotel_Lat,
                               'Longitude': Hotel_Lng,
                                'Adress': Hotel_Adresses,
                            
                                })
Best_Hotel_data.to_csv('output_data\Best_Hotel_Data.csv')
Best_Hotel_data


,Country,Humidity,City,Name,Latitude,Longitude,Adress
0,Turkmenistan,70,Balkanabat,Nebitchi Hotel,39.514673,54.359896,Balkanabat
1,New Caledonia,45,Voh,Le gîte du Koniambo,-20.983312,164.722214,Lot 1 pie section Koniambo Nord Ouest BP : 653...
2,Kazakhstan,65,Komsomolets,Gostinitsa Zhemchuzhina,53.751113,62.054467,Karabalyk
3,United States,47,Laguna,Hilton Garden Inn Sacramento Elk Grove,38.422528,-121.397573,"9241 Laguna Springs Dr, Elk Grove"
4,Egypt,25,Aswān,Sofitel Legend Old Cataract Aswan,24.094242,32.897351,"Abtal El Tahrir, Street"
5,Queensland,46,Townsville,Mercure Townsville,-19.271492,146.785922,"166 Woolcock St, Townsville"
6,Spain,88,Puerto del Rosario,Hotel El Mirador,28.473816,-13.865657,"Ctra. Playa Blanca, 45, Puerto del Rosario"
7,Libya,71,Sirte,City Hotel,31.206438,16.588910,رقم 1، Sirte
8,China,50,Linxia Chengguanzhen,Hehuang Mingzhu International Hotel,35.598524,103.245118,"Linxia, Linxia Hui Autonomous Prefecture, Nanb..."
9,Brazil,57,São João dos Patos,HOTEL SÃO CRISTOVÃO,-6.498315,-43.701501,"R. João Pessoa, 175, São João dos Patos"


In [8]:
# Readig the Best hotel information from CSV File 

Best_Hotel_data=pd.read_csv('output_data\Best_Hotel_Data.csv')
Best_Hotel_data
# Best_Hotel_City_List=Best_Hotel_data['City']
# Best_Hotel_Countries=city_data.loc[city_data['City'].isin(Best_Hotel_City_List),city_data['']]


,Unnamed: 0,Country,Humidity,City,Name,Latitude,Longitude,Adress
0,0,Turkmenistan,70,Balkanabat,Nebitchi Hotel,39.514673,54.359895,Balkanabat
1,1,New Caledonia,45,Voh,Le gîte du Koniambo,-20.983312,164.722214,Lot 1 pie section Koniambo Nord Ouest BP : 653...
2,2,Kazakhstan,65,Komsomolets,Gostinitsa Zhemchuzhina,53.751113,62.054467,Karabalyk
3,3,United States,47,Laguna,Hilton Garden Inn Sacramento Elk Grove,38.422528,-121.397573,"9241 Laguna Springs Dr, Elk Grove"
4,4,Egypt,25,Aswān,Sofitel Legend Old Cataract Aswan,24.094242,32.897351,"Abtal El Tahrir, Street"
5,5,Queensland,46,Townsville,Mercure Townsville,-19.271492,146.785922,"166 Woolcock St, Townsville"
6,6,Spain,88,Puerto del Rosario,Hotel El Mirador,28.473816,-13.865657,"Ctra. Playa Blanca, 45, Puerto del Rosario"
7,7,Libya,71,Sirte,City Hotel,31.206438,16.588910,رقم 1، Sirte
8,8,China,50,Linxia Chengguanzhen,Hehuang Mingzhu International Hotel,35.598524,103.245118,"Linxia, Linxia Hui Autonomous Prefecture, Nanb..."
9,9,Brazil,57,São João dos Patos,HOTEL SÃO CRISTOVÃO,-6.498315,-43.701501,"R. João Pessoa, 175, São João dos Patos"


In [18]:
# NOTE: Do not change any of the code in this cell
Names_List=Best_Hotel_data['Name']
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Adress</dt><dd>{Adress}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Best_Hotel_data.iterrows()]


#Best_Hotels Locations
locations = Best_Hotel_data[['Latitude', 'Longitude']]


#Create Best Hotels Markers Map
fig = gmaps.figure(center=(0,0),zoom_level=1.5)


markers = gmaps.marker_layer(locations,info_box_template,
    info_box_content=hotel_info)
fig.add_layer(markers)




# Fill NaN values and convert to float
rating1 = city_data["Humidity"].astype(int)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations1, weights=rating1,
                                dissipating=False, max_intensity=100,
                                 point_radius=2,opacity =.8)


# Add layer


fig.add_layer(heat_layer)

# Display figure
fig






Figure(layout=FigureLayout(height='420px'))